Let's get to the point.
I'm refactoring old code to fit into the old models that I made
The problem is the preprocessing
And as always my crutch on AI has lead me astray
Regardless, I think I can get through this?

This notebook was written based on these resources:
https://www.kaggle.com/code/someadityamandal/bitcoin-time-series-forecasting/notebook
https://github.com/dataquestio/project-walkthroughs/tree/master/bitcoin_price
https://www.geeksforgeeks.org/time-series-forecasting-using-tensorflow/

Special thank you to David for, as always, having the best resources.

In [ ]:
# Let's try all that preprocessing again.
import numpy as np
import pandas as pd
import keras.layers as layers
import keras.models as models

import datetime, pytz
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# import os
# <!-- print(os.listdir('./data')) -->

In [27]:
# Okay so for some reasons those imports were taking WAY too long.

import yfinance as yf
import os

btc_ticker = yf.Ticker('BTC-USD')

if os.path.exists('btc.csv'):
    btc = pd.read_csv('btc.csv', index_col=0)
else:
    btc=btc_ticker.history(perid='max')
    btc.to_csv('btc.csv')

print(btc.head())
btc.info()

                                 Open        High         Low       Close  \
Date                                                                        
2014-09-17 00:00:00+00:00  465.864014  468.174011  452.421997  457.334015   
2014-09-18 00:00:00+00:00  456.859985  456.859985  413.104004  424.440002   
2014-09-19 00:00:00+00:00  424.102997  427.834991  384.532013  394.795990   
2014-09-20 00:00:00+00:00  394.673004  423.295990  389.882996  408.903992   
2014-09-21 00:00:00+00:00  408.084991  412.425995  393.181000  398.821014   

                             Volume  Dividends  Stock Splits  
Date                                                          
2014-09-17 00:00:00+00:00  21056800        0.0           0.0  
2014-09-18 00:00:00+00:00  34483200        0.0           0.0  
2014-09-19 00:00:00+00:00  37919700        0.0           0.0  
2014-09-20 00:00:00+00:00  36863600        0.0           0.0  
2014-09-21 00:00:00+00:00  26580100        0.0           0.0  
<class 'pandas.core

In [28]:
btc.index = pd.to_datetime(btc.index)
btc
btc.drop(columns=["Dividends", "Stock Splits"], inplace=True)
btc.columns = [c.lower() for c in btc.columns]
print(btc.head())
btc.info()

                                 open        high         low       close  \
Date                                                                        
2014-09-17 00:00:00+00:00  465.864014  468.174011  452.421997  457.334015   
2014-09-18 00:00:00+00:00  456.859985  456.859985  413.104004  424.440002   
2014-09-19 00:00:00+00:00  424.102997  427.834991  384.532013  394.795990   
2014-09-20 00:00:00+00:00  394.673004  423.295990  389.882996  408.903992   
2014-09-21 00:00:00+00:00  408.084991  412.425995  393.181000  398.821014   

                             volume  
Date                                 
2014-09-17 00:00:00+00:00  21056800  
2014-09-18 00:00:00+00:00  34483200  
2014-09-19 00:00:00+00:00  37919700  
2014-09-20 00:00:00+00:00  36863600  
2014-09-21 00:00:00+00:00  26580100  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3713 entries, 2014-09-17 00:00:00+00:00 to 2024-11-16 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---

Not going to lie
I'm ashamed I didn't do that print earlier.
Regardless, easy bug to fix.

In [29]:
btc['tomorrow'] = btc['close'].shift(-1)
btc['target']  = (btc['tomorrow'] > btc['close']).astype(int)
btc.dropna(inplace=True)

split_date = "2024-05-05"
data_train = btc.loc[btc.index <= split_date].copy()
data_test = btc.loc[btc.index > split_date].copy()

In [31]:
sc = MinMaxScaler()
train_scaled = sc.fit_transform(data_train[['close']])
test_scaled = sc.transform(data_test[['close']])

X_train = train_scaled[:-1]
y_train = train_scaled[1:]

X_test = test_scaled[:-1]
y_test = test_scaled[1:]

X_train = np.reshape(X_train, (len(X_train), 1, 1))
X_test = np.reshape(X_test, (len(X_test), 1, 1))

In [32]:
# please, please, please work
model1 = models.Sequential()

model1.add(layers.LSTM(128, activation='relu', input_shape=(1, 1)))
model1.add(layers.Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

model2 = models.Sequential()

# This one's more promising than the other's I've seen
model2.add(layers.LSTM(100, return_sequences=True))
model2.add(layers.LSTM(100, return_sequences=True))
model2.add(layers.LSTM(50))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1, activation= 'linear'))
# Interesting that this one does two different activation functions
# Suppose that makes sense though

# Ok last one is that weird amalgamation I did last time.
# Copy and pasting isn't cheating when it's your own code
model3 = models.Sequential()

model3.add(layers.LSTM(128, activation='relu', input_shape=(1, 1)))
model3.add(layers.Dropout(0.2))
model3.add(layers.Dense(1))
model3.add(layers.Dropout(0.2))
model3.add(layers.Dense(1))
model3.compile(loss='mean_squared_error', optimizer='adam')
model3.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

model1.compile(loss='mean_squared_error', optimizer='adam')
model2.compile(loss='mean_squared_error', optimizer='adam')
model3.compile(loss='mean_squared_error', optimizer='adam')

model1.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)
model2.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)
model3.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
71/71 - 6s - 89ms/step - loss: 0.0681
Epoch 2/100
71/71 - 0s - 7ms/step - loss: 0.0296
Epoch 3/100
71/71 - 0s - 6ms/step - loss: 0.0130
Epoch 4/100
71/71 - 0s - 7ms/step - loss: 0.0025
Epoch 5/100
71/71 - 0s - 4ms/step - loss: 4.2741e-04
Epoch 6/100
71/71 - 0s - 4ms/step - loss: 3.1364e-04
Epoch 7/100
71/71 - 0s - 4ms/step - loss: 3.0262e-04
Epoch 8/100
71/71 - 0s - 5ms/step - loss: 2.9651e-04
Epoch 9/100
71/71 - 0s - 4ms/step - loss: 2.8899e-04
Epoch 10/100
71/71 - 0s - 3ms/step - loss: 2.8275e-04
Epoch 11/100
71/71 - 0s - 3ms/step - loss: 2.7635e-04
Epoch 12/100
71/71 - 0s - 3ms/step - loss: 2.6762e-04
Epoch 13/100
71/71 - 0s - 3ms/step - loss: 2.6114e-04
Epoch 14/100
71/71 - 0s - 4ms/step - loss: 2.5374e-04
Epoch 15/100
71/71 - 0s - 3ms/step - loss: 2.4458e-04
Epoch 16/100
71/71 - 0s - 3ms/step - loss: 2.3922e-04
Epoch 17/100
71/71 - 0s - 4ms/step - loss: 2.3084e-04
Epoch 18/100
71/71 - 0s - 4ms/step - loss: 2.2243e-04
Epoch 19/100
71/71 - 0s - 3ms/step - loss: 2.1540e-0

HOLY HECKING CHRISTMAS CRACKERS IT'S WORKING
A bit faster than it was before though. So that's a wee bit concerning
Like. Really concerning actually

In [33]:
model1.summary()
model2.summary()
model3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,069 (781.52 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,075 (781.55 KB)

 Trainable params: 66,691 (260.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,384 (521.04 KB)

In [35]:
predictions1 = model1.predict(X_test)
predictions2 = model2.predict(X_test)
predictions3 = model3.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  


I /really/ feel like those should have taken longer
I'm getting worried